# Day 20: Trench Map

[https://adventofcode.com/2021/day/20](https://adventofcode.com/2021/day/20)

## Description

### Part One

With the scanners fully deployed, you turn their attention to mapping the floor of the ocean trench.

When you get back the image from the scanners, it seems to just be random noise. Perhaps you can combine an image enhancement algorithm and the input image (your puzzle input) to clean it up a little.

For example:

    ..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##
    #..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###
    .######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.
    .#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....
    .#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..
    ...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....
    ..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

    #..#.
    #....
    ##..#
    ..#..
    ..###


The first section is the _image enhancement algorithm_. It is normally given on a single line, but it has been wrapped to multiple lines in this example for legibility. The second section is the _input image_, a two-dimensional grid of _light pixels_ (`#`) and _dark pixels_ (`.`).

The image enhancement algorithm describes how to enhance an image by _simultaneously_ converting all pixels in the input image into an output image. Each pixel of the output image is determined by looking at a 3x3 square of pixels centered on the corresponding input image pixel. So, to determine the value of the pixel at (5,10) in the output image, nine pixels from the input image need to be considered: (4,9), (4,10), (4,11), (5,9), (5,10), (5,11), (6,9), (6,10), and (6,11). These nine input pixels are combined into a single binary number that is used as an index in the _image enhancement algorithm_ string.

For example, to determine the output pixel that corresponds to the very middle pixel of the input image, the nine pixels marked by `[...]` would need to be considered:

    # . . # .
    #[. . .].
    #[# . .]#
    .[. # .].
    . . # # #


Starting from the top-left and reading across each row, these pixels are `...`, then `#..`, then `.#.`; combining these forms `...#...#.`. By turning dark pixels (`.`) into `0` and light pixels (`#`) into `1`, the binary number `000100010` can be formed, which is `34` in decimal.

The image enhancement algorithm string is exactly 512 characters long, enough to match every possible 9-bit binary number. The first few characters of the string (numbered starting from zero) are as follows:

    0         10        20        30  34    40        50        60        70
    |         |         |         |   |     |         |         |         |
    ..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##


In the middle of this first group of characters, the character at index 34 can be found: `#`. So, the output pixel in the center of the output image should be `#`, a _light pixel_.

This process can then be repeated to calculate every pixel of the output image.

Through advances in imaging technology, the images being operated on here are _infinite_ in size. _Every_ pixel of the infinite output image needs to be calculated exactly based on the relevant pixels of the input image. The small input image you have is only a small region of the actual infinite input image; the rest of the input image consists of dark pixels (`.`). For the purposes of the example, to save on space, only a portion of the infinite-sized input and output images will be shown.

The starting input image, therefore, looks something like this, with more dark pixels (`.`) extending forever in every direction not shown here:

    ...............
    ...............
    ...............
    ...............
    ...............
    .....#..#......
    .....#.........
    .....##..#.....
    .......#.......
    .......###.....
    ...............
    ...............
    ...............
    ...............
    ...............


By applying the image enhancement algorithm to every pixel simultaneously, the following output image can be obtained:

    ...............
    ...............
    ...............
    ...............
    .....##.##.....
    ....#..#.#.....
    ....##.#..#....
    ....####..#....
    .....#..##.....
    ......##..#....
    .......#.#.....
    ...............
    ...............
    ...............
    ...............


Through further advances in imaging technology, the above output image can also be used as an input image! This allows it to be enhanced _a second time_:

    ...............
    ...............
    ...............
    ..........#....
    ....#..#.#.....
    ...#.#...###...
    ...#...##.#....
    ...#.....#.#...
    ....#.#####....
    .....#.#####...
    ......##.##....
    .......###.....
    ...............
    ...............
    ...............


Truly incredible - now the small details are really starting to come through. After enhancing the original input image twice, _`35`_ pixels are lit.

Start with the original input image and apply the image enhancement algorithm twice, being careful to account for the infinite size of the images. _How many pixels are lit in the resulting image?_

In [2]:
from collections import Counter
from typing import Generator


def load(filename: str) -> tuple[str, list[str]]:
    with open(filename) as f:
        rules = f.readline().strip()

        data = [line.strip() for line in f if line.strip()]
    return rules, data


def neighbours(x: int, y: int) -> Generator[tuple[int, int], None, None]:
    yield from [(xn, yn) for yn in (y - 1, y, y + 1) for xn in (x - 1, x, x + 1)]


def bounds(grid: dict[tuple[int, int], str]) -> tuple[int, int, int, int]:
    left = min(x for x, _ in grid) - 1
    right = max(x for x, _ in grid) + 1
    top = min(y for _, y in grid) - 1
    bottom = max(y for _, y in grid) + 1
    return left, top, right, bottom


def algorithm(rules: str, grid: dict[tuple[int, int], str], default: str):
    new_grid: dict[tuple[int, int], str] = {}

    left, top, right, bottom = bounds(grid)
    for x in range(left, right + 1):
        for y in range(top, bottom + 1):
            index = int(
                "".join(grid.get((x1, y1), default) for x1, y1 in neighbours(x, y)), 2
            )
            if rules[index] == "#":
                new_grid[x, y] = "1"
            else:
                new_grid[x, y] = "0"
    return new_grid


def reconstruct(grid: dict[tuple[int, int], str]) -> str:
    left, top, right, bottom = bounds(grid)
    rows: list[str] = []
    for y in range(top, bottom):
        rows.append("".join(grid.get((x, y), " ") for x in range(left, right)))
    return "\n".join(rows)


def part1(filename: str, nsteps: int = 2) -> int:
    rules, raw = load(filename)
    grid = {
        (x, y): ("1" if c == "#" else "0")
        for y, row in enumerate(raw)
        for x, c in enumerate(row)
    }

    grid = algorithm(rules, grid, "0")
    default = "1" if rules[0] == "#" else "0"
    for n in range(nsteps - 1):
        grid = algorithm(rules, grid, default)
        dc = rules[0] if default == "0" else rules[511]
        default = "1" if dc == "#" else "0"
    counts = Counter(grid.values())
    return counts["1"]


assert part1("input/day20-test.txt") == 35
print("Part 1", part1("input/day20.txt"))

Part 1 5231

**Your puzzle answer was 5231.**

### Part Two

You still can't quite make out the details in the image. Maybe you just didn't [enhance](https://en.wikipedia.org/wiki/Kernel_(image_processing)) it <span title="Yeah, that's definitely the problem.">enough</span>.

If you enhance the starting input image in the above example a total of _50_ times, _`3351`_ pixels are lit in the final output image.

Start again with the original input image and apply the image enhancement algorithm 50 times. _How many pixels are lit in the resulting image?_

In [3]:
part2 = part1
assert part2("input/day20-test.txt", 50) == 3351
print("Part 2", part2("input/day20.txt", 50))

Part 2 14279

**Your puzzle answer was 14279.**
<div class="alert alert-info">Nice little twist there in part 2 where the infinite grid flashes alternately 1s and 0s</div>
